In [4]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from math import exp

x=pd.read_csv(".\train_data (1).csv")
y=pd.read_csv(".\train_labels.csv")

x=x.dropna()
y=y.dropna()

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x,y, test_size=0.2, random_state=275)

X_train=np.array(X_train)

y_train=np.array(y_train)

X_val=np.array(X_val)

y_val=np.array(y_val)

## Calculate z=wTx+b for a perceptron
#Takes w and X and returns z=wTx+b for it
def z(w, x):
#activation is set to bias (last element of w) initially
    z = w[-1]
#only iterate over weights that need to be multiplied
    for i in range(len(w)-1):
        z = z + (w[i] * x[i])
#print(z)
    return z

# sigmoid activation function
def sigmoid(z):
    a = 1.0 / (1.0 + exp(-z))
    return a

# Derivative of perceptron output
def derv_sigmoid(output):
    d = output * (1.0 - output)
    return d

# MLP
##taken in number of inputs, perceptrons in hidden layer and number of outputs
def MLP(n_inputs, n_hidden, n_outputs):
#define MLP as a list with layers as elements
    mlp = list()
# initialize all weights of hidden layer to 0 for every perceptron in it
    hidden_layer = [ {'w':[0 for i in range(n_inputs + 1)]}
                    for j in range(n_hidden)]
    mlp.append(hidden_layer)
#initialize weights of output layer to 0 for every output node 
    output_layer = [{'w' : [0 for i in range(n_hidden + 1)]}
                    for j in range(n_outputs)]
    mlp.append(output_layer)
    return mlp


# Forward propagation
def forward_prop(mlp, input_entry):
    x = input_entry
    for layer in mlp:
        x_new = []
        for perceptron in layer:
#Find z=wT+b for the perceptron
            z1 = z(perceptron['w'], x)
#Find output of sigmoid for perceptron
            perceptron['output']= sigmoid(z1)
            x_new.append(perceptron['output'])
        x = x_new
    return x

# Backpropagation algo
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
#intialize error as a list
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['w'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] *derv_sigmoid(neuron['output'])

            
# Update network weights with error
def update_w(mlp, input_entry, lr):
    for i in range(len(mlp)):
#Starting updates from last column
        x = input_entry[:-1]
        if i != 0:
            x = [perceptron['output'] for perceptron in mlp[i - 1]]
        for perceptron in mlp[i]:
            for j in range(len(x)):
                perceptron['w'][j]= perceptron['w'][j]+ lr*perceptron['delta']*x[j]
            perceptron['w'][-1] =perceptron['w'][-1]+ lr * perceptron['delta']

output_record=[]
def train(mlp, x_train, lr, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        error = 0
        for n_row in range (x_train.shape[0]):
            outputs = forward_prop(mlp, x_train[n_row])
            output_record.append(outputs)
            y = y_train[n_row]
            error = error + sum([(y[i]-outputs[i])**2 for i in range(len(y))])
            backward_propagate_error(mlp, y)
            update_w(mlp, x_train[n_row], lr)    
        print('train network','>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, lr, error))
        

n_inputs = 784
n_outputs = 4
n_hidden=1
network = MLP(n_inputs, 10, n_outputs)
train(network, X_train, 0.2, 10, n_outputs)


import pickle
pkl_filename = "pickle_model1.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(network, file)
    
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)   

    
# Make a prediction with a network
def predict(network, X):
    arr2=[]
    for row in range (X.shape[0]):
        outputs = forward_prop(network, X[row])
        a=outputs.index(max(outputs))
        arr2.append(a)
    arr2=np.array(arr2)
    b = np.zeros((arr2.size, arr2.max()+1))
    b[np.arange(arr2.size),arr2] = 1
    return b

Ypredict =predict(pickle_model,X_val)
print('Expected=', y_val)
print('Got=',  Ypredict)
    
#prediction = predict(network, X_val)

def accuracy(y_true, y_pred):
    if not (len(y_true) == len(y_pred)):
        print('Size of predicted and true labels not equal.')
        return 0.0

    corr = 0
    for i in range(0,len(y_true)):
        corr += 1 if (y_true[i] == y_pred[i]).all() else 0

    return corr/len(y_true)

print('accuracy:',accuracy(y_val,Ypredict))



[{'w': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,